In [2]:
import wave
import numpy as np

# read file 
FPB = 2048
file_path = "example.wav"
# 读取本地文件
with wave.open(file_path, 'rb') as wf:
    signal = wf.readframes(FPB)
    sound = np.frombuffer(signal, dtype=np.int16).astype(np.float32) / 32768

print("sound shape: ", sound.shape)

Read success
sound shape:  (2048,)


In [4]:
import soundfile as sf
import rir_generator as rir
import scipy.signal as ss
import numpy as np
import wave

def add_room_reverb(audio_path, save_path, 
                   room_dim=[5, 4, 6],
                   source_pos=[2, 3.5, 2],
                   receiver_pos=[2, 1.5, 2],
                   rt60=0.4):
    """
    添加基于房间模型的混响效果
    
    参数:
    - audio_path: 输入音频文件路径
    - save_path: 输出音频文件路径
    - room_dim: 房间尺寸 [x y z] (米)
    - source_pos: 声源位置 [x y z] (米)
    - receiver_pos: 接收器位置 [x y z] (米)
    - rt60: 混响时间(秒)
    """
    # 使用 wave 读取音频
    with wave.open(audio_path, 'rb') as wav:
        # 获取音频参数
        fs = wav.getframerate()
        n_channels = wav.getnchannels()
        sampwidth = wav.getsampwidth()
        n_frames = wav.getnframes()
        
        # 读取音频数据并转换为numpy数组
        signal = wav.readframes(n_frames)
        signal = np.frombuffer(signal, dtype=np.int16)
        
        # 重塑数组为(n_samples, n_channels)
        signal = signal.reshape(-1, n_channels)
        # 转换为float类型进行处理
        signal = signal.astype(np.float32) / 32768.0

    # 生成房间冲击响应
    h = rir.generate(
        c=340,                      # 声速 (m/s)
        fs=fs,                      # 采样率
        r=[receiver_pos],           # 接收器位置
        s=source_pos,               # 声源位置
        L=room_dim,                 # 房间尺寸
        reverberation_time=rt60,    # 混响时间
        nsample=4096                # 输出样本数
    )
    
    # 确保h形状正确 (nsample, 1)
    h = h.reshape(-1, 1)
    
    # 对每个声道进行卷积
    reverbed = np.zeros_like(signal)
    for i in range(signal.shape[1]):
        reverbed[:, i] = ss.convolve(signal[:, i], h[:, 0], mode='same')
    
    # 归一化输出
    reverbed = reverbed / np.max(np.abs(reverbed))
    
    # 保存处理后的音频
    sf.write(save_path, reverbed, fs)
    
    return reverbed, fs


add_room_reverb("example.wav", "example_reverbed.wav")

(array([[-2.8061587e-04],
        [-1.7931614e-03],
        [-2.2212041e-03],
        ...,
        [-5.8562870e-05],
        [-4.4033848e-05],
        [-4.7326383e-05]], dtype=float32),
 8000)

In [6]:
import pyaudio

def add_room_reverb(input_signal, fs,
                   room_dim=[5.0, 4.0, 6.0],
                   source_pos=[2.0, 3.5, 2.0],
                   receiver_pos=[2.0, 1.5, 2.0],
                   rt60=0.4):
    h = rir.generate(
        c=340,
        fs=fs,
        r=[receiver_pos],
        s=source_pos,
        L=room_dim,
        reverberation_time=rt60,
        nsample=4096
    )
    h = h.reshape(-1, 1)
    reverbed = ss.convolve(input_signal, h[:, 0], mode='same')
    max_val = np.max(np.abs(reverbed))
    if max_val > 1e-6:
        reverbed = reverbed / max_val
    else:
        reverbed = np.zeros_like(reverbed)
    return reverbed


audio_path = "example.wav"
with wave.open(audio_path, 'rb') as wav:
    # 获取音频参数
    fs = wav.getframerate()
    n_channels = wav.getnchannels()
    sampwidth = wav.getsampwidth()
    n_frames = wav.getnframes()
    
    # 读取音频数据并转换为numpy数组
    signal = wav.readframes(n_frames)
    signal = np.frombuffer(signal, dtype=np.int16)
    
    # 重塑数组为(n_samples, n_channels)
    signal = signal.reshape(-1, n_channels)
    # 转换为float类型进行处理
    signal = signal.astype(np.float32) / 32768.0

reverbed = add_room_reverb(signal[:, 0], fs)


# 准备音频播放参数
CHANNELS = 1
RATE = fs
WIDTH = 2  # 16位采样
FPB = 1024  # 每个缓冲区的帧数
# 将float32转换回int16进行播放
audio_data = (reverbed * 32767).astype(np.int16)

# 初始化PyAudio
p = pyaudio.PyAudio()

# 打开音频流
stream = p.open(
    format=p.get_format_from_width(WIDTH),
    channels=CHANNELS,
    rate=RATE,
    input=False,
    output=True,
    frames_per_buffer=FPB
)

# 播放音频
try:
    stream.write(audio_data.tobytes())
finally:
    # 关闭音频流
    stream.stop_stream()
    stream.close()
    p.terminate()

In [1]:
from pedalboard import Pedalboard, Reverb
from pedalboard.io import AudioFile

# Make a Pedalboard object, containing multiple audio plugins:
board = Pedalboard([Reverb(room_size=0.25)])

# Open an audio file for reading, just like a regular file:
with AudioFile('example.wav') as f:

  # Open an audio file to write to:
  with AudioFile('output.wav', 'w', f.samplerate, f.num_channels) as o:

    # Read one second of audio at a time, until the file is empty:
    while f.tell() < f.frames:
      chunk = f.read(f.samplerate)
      print(type(chunk))
      print(chunk.shape)
      print(f.samplerate)
      # Run the audio through our pedalboard:
      effected = board(chunk, f.samplerate, reset=False)

      # Write the output to our output file:
      o.write(effected)

<class 'numpy.ndarray'>
(1, 8000)
8000
<class 'numpy.ndarray'>
(1, 8000)
8000
<class 'numpy.ndarray'>
(1, 8000)
8000
<class 'numpy.ndarray'>
(1, 3200)
8000


In [ ]:
import pyaudio

p = pyaudio.PyAudio()

# 找到麦克风设备索引
# 您需要根据设备名称判断，下面的例子只展示基本方法
device_index = None
for i in range(p.get_device_count()):
    info = p.get_device_info_by_index(i)
    print(f"Device {i}: {info.get('name')}, Input channels: {info.get('maxInputChannels')}")
    # 假设找到名称中包含"mic"的设备作为麦克风
    if "mic" in info.get('name').lower():
        device_index = i

if device_index is None:
    print("未找到麦克风设备")
else:
    test_rates = [8000, 16000, 22050, 32000, 44100, 48000, 96000]
    
    for rate in test_rates:
        try:
            if p.is_format_supported(rate,
                                     input_device=device_index,
                                     input_channels=1,
                                     input_format=pyaudio.paFloat32):
                print(f"支持采样率: {rate} Hz")
            else:
                print(f"不支持采样率: {rate} Hz")
        except ValueError:
            print(f"不支持采样率: {rate} Hz")

p.terminate()

In [2]:
def add_reverb_pedalboard(audio_path, save_path, room_size=0.8, wet_level=0.8):
    """使用 Pedalboard 添加混响效果
    Args:
        audio_path (str): 输入音频路径
        save_path (str): 输出音频路径
        room_size (float): 房间大小 (0-1)
        wet_level (float): 湿信号比例 (0-1)
    """
    from pedalboard import Pedalboard, Reverb
    import wave
    import numpy as np

    # 读取wave文件
    with wave.open(audio_path, 'rb') as wf:
        # 获取音频参数
        channels = wf.getnchannels()
        sample_width = wf.getsampwidth()
        sample_rate = wf.getframerate()
        n_frames = wf.getnframes()
        
        # 读取音频数据
        audio_data = wf.readframes(n_frames)
        audio_array = np.frombuffer(audio_data, dtype=np.int16)
        
        # 转换为float32格式 (-1 到 1)
        audio_float = audio_array.astype(np.float32) / 32768.0
        
        # 重塑数组以匹配声道
        if channels == 2:
            audio_float = audio_float.reshape(-1, 2)
        else:
            audio_float = audio_float.reshape(-1, 1)

    # 创建效果器
    board = Pedalboard([Reverb(
        room_size=room_size,
        wet_level=wet_level
    )])

    # 应用效果
    effected = board(audio_float, sample_rate)

    # 转换回int16格式
    audio_int = (effected * 32768).astype(np.int16)

    # 保存处理后的音频
    with wave.open(save_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(sample_rate)
        wf.writeframes(audio_int.tobytes())

    return audio_int


add_reverb_pedalboard("example.wav", "2222_reverbed.wav")

array([[  353],
       [  510],
       [  494],
       ...,
       [-1549],
       [-1559],
       [-1827]], dtype=int16)

In [17]:
import wave
import numpy as np
from pedalboard import Pedalboard, Reverb

def process_audio_by_frames(input_path, output_path, frame_size=2048*100, room_size=0.8, wet_level=0.8):
    # 创建效果器
    board = Pedalboard([Reverb(room_size=room_size, wet_level=wet_level)])
    
    with wave.open(input_path, 'rb') as wf_in:
        # 获取音频参数
        channels = wf_in.getnchannels()
        sample_width = wf_in.getsampwidth()
        sample_rate = wf_in.getframerate()
        
        with wave.open(output_path, 'wb') as wf_out:
            # 设置输出文件参数
            wf_out.setnchannels(channels)
            wf_out.setsampwidth(sample_width)
            wf_out.setframerate(sample_rate)
            
            # 逐帧处理
            while True:
                # 读取一帧数据
                audio_data = wf_in.readframes(frame_size)
                if not audio_data:
                    break
                    
                # 转换为numpy数组
                audio_array = np.frombuffer(audio_data, dtype=np.int16)
                
                # 转换为float32
                audio_float = audio_array.astype(np.float32) / 32768.0
                
                # 重塑数组
                if channels == 2:
                    audio_float = audio_float.reshape(-1, 2)
                else:
                    audio_float = audio_float.reshape(-1, 1)
                
                # 应用效果
                effected = board(audio_float, sample_rate)
                
                # 转回int16
                audio_int = (effected * 32768).astype(np.int16)
                
                # 写入输出文件
                wf_out.writeframes(audio_int.tobytes())

# 使用示例
process_audio_by_frames('222.wav', 'output_222.wav', frame_size=1024)

In [ ]:
from pedalboard import Pedalboard, Reverb
from pedalboard.io import AudioFile

# Make a Pedalboard object, containing multiple audio plugins:
board = Pedalboard([Reverb(room_size=0.25)])

# Open an audio file for reading, just like a regular file:
with AudioFile('222.wav') as f:

  # Open an audio file to write to:
  with AudioFile('output.wav', 'w', f.samplerate, f.num_channels) as o:

    # Read one second of audio at a time, until the file is empty:
    while f.tell() < f.frames:
      chunk = f.read(f.samplerate)

      # Run the audio through our pedalboard:
      effected = board(chunk, f.samplerate, reset=False)

      # Write the output to our output file:
      o.write(effected)

In [22]:
import pyaudio
import numpy as np
from pedalboard import Pedalboard, Reverb

class RealtimeReverb:
    def __init__(self):
        # 音频参数
        self.CHUNK = 1024
        self.FORMAT = pyaudio.paFloat32
        self.CHANNELS = 1
        self.RATE = 44100
        
        # 创建效果器
        self.board = Pedalboard([
            Reverb(
                room_size=0.8,
                wet_level=0.7,
                damping=0.5,
                width=1.0
            )
        ])
        
        # 初始化PyAudio
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.RATE,
            input=True,
            output=True,
            frames_per_buffer=self.CHUNK,
            stream_callback=self.audio_callback
        )
        
    def audio_callback(self, in_data, frame_count, time_info, status):
        try:
            # 转换输入数据
            audio_data = np.frombuffer(in_data, dtype=np.float32)
            
            # 重塑数组
            if audio_data.ndim == 1:
                audio_data = audio_data.reshape(-1, 1)
            
            # 应用效果
            processed = self.board(audio_data, self.RATE)
            
            return (processed.tobytes(), pyaudio.paContinue)
        except Exception as e:
            print(f"处理错误: {str(e)}")
            return (in_data, pyaudio.paContinue)
            
    def start(self):
        self.stream.start_stream()
        print("开始处理...")
        try:
            while self.stream.is_active():
                import time
                time.sleep(0.1)
        except KeyboardInterrupt:
            print("停止处理")
        finally:
            self.stop()
            
    def stop(self):
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()

if __name__ == "__main__":
    processor = RealtimeReverb()
    try:
        processor.start()
    except Exception as e:
        print(f"运行错误: {str(e)}")

开始处理...
停止处理


In [25]:
import tempfile
import wave
import numpy as np
from pedalboard import Pedalboard, Reverb
from pedalboard.io import AudioFile
import pyaudio

board = Pedalboard([Reverb(room_size=0.8, wet_level=0.7, damping=0.5, width=1.0)])

def process_audio_chunk(in_data, frame_count, time_info, status):
    """使用 AudioFile 处理音频数据"""
    audio_data = np.frombuffer(in_data, dtype=np.float32)

    # 创建临时文件存储音频块
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_in, \
            tempfile.NamedTemporaryFile(suffix='.wav') as temp_out:
        
        # 保存输入数据到临时文件
        with wave.open(temp_in.name, 'wb') as wf:
            wf.setnchannels(1)
            wf.setsampwidth(4)  # float32
            wf.setframerate(44100)
            wf.writeframes(audio_data)
        
        # 使用 AudioFile 处理
        with AudioFile(temp_in.name) as f:
            with AudioFile(temp_out.name, 'w', f.samplerate, f.num_channels) as o:
                while f.tell() < f.frames:
                    chunk = f.read(f.samplerate)
                    # Run the audio through our pedalboard:
                    effected = board(chunk, f.samplerate, reset=False)
                    # Write the output to our output file:
                    o.write(effected)
        
        # 读取处理后的数据
        with wave.open(temp_out.name, 'rb') as wf:
            processed_data = wf.readframes(wf.getnframes())
            
        return (processed_data, pyaudio.paContinue)
    
def main():
    p = pyaudio.PyAudio()
    
    stream = p.open(
        format=pyaudio.paFloat32,
        channels=1,
        rate=44100,
        input=True,
        output=True,
        frames_per_buffer=1024,
        stream_callback=process_audio_chunk
    )
    
    print("* 开始录音和实时处理. 按 Ctrl+C 停止.")
    
    try:
        while stream.is_active():
            stream.start_stream()
    except KeyboardInterrupt:
        print("* 停止录音")
    finally:
        stream.stop_stream()
        stream.close()
        p.terminate()

if __name__ == "__main__":
    main()

* 开始录音和实时处理. 按 Ctrl+C 停止.
